In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Sample from an NFW profile ##

Dark matter halos tend to have a characteristic shape, or density profile.  There are several fitting functions available, and much ink has been spilled about the relative merits of each.  The most commonly adopted is a double-power-law form, known as the [NFW profile](https://en.wikipedia.org/wiki/Navarro%E2%80%93Frenk%E2%80%93White_profile) after the initials of its authors:
$$
  \rho(r) \propto x^{-1}(1+x)^{-2}\qquad x\equiv \frac{r}{r_s}
$$
where $x$ is a dimensionless radius and $r_s$ is the scale radius of the halo defined by the point where the log-slope is $-2$.  This form has two free parameters: the normalization and $r_s$.  However traditionally we use the total mass, $M_{\rm vir}$, and concentration, $c=r_{\rm vir}/r_s$, instead.  Since the profile doesn't integrate to finite mass the choice of $M_{\rm vir}$ is conventional.  While many choices are possible, a sensible choice (motivated by spherical top-hat collapse) is the mass within which the mean density is $200\times$ the background density of the Universe.  It then follows by definition that $M_{\rm vir} = (800\pi/3)\bar{\rho}_m\,r_{\rm vir}^3$.  An alternative is to use the critical density instead of the mean density to define $r_{\rm vir}$.  Other conventions are also adopted.  While a full understanding of these universal profiles still eludes us, many
aspects of halo formation may be understood from fairly general considerations of gravitational collapse.

It is cosmologically interesting that $M$ and $c$ tend to be correlated but for our purposes now we'll just assume $M$ and $c$ are known.  Physically, the inner, $r^{−1}$, part of the halo forms early, and then $r_s$ stays pretty constant as subsequently accreted dark matter is mostly kept away from the center by the angular momentum barrier. Thus the halo concentration, $c\equiv r_{\rm vir}/r_s$, grows with time; simulations show that $c$ typically grows (roughly) linearly with scale factor: $a = (1+z)^{−1}$.  The average mass accretion history of halos is (approximately) exponential in redshift, $z$, and the angular momentum parameter, $\lambda$, of the halo typically grows significantly in halo major mergers (i.e., mergers with mass ratios between unity and 1/3) and declines as mass is accreted in minor mergers. Dark matter halos are generally triaxial spheroids; they are more elongated at smaller radii, larger redshifts, and higher masses (perhaps reflecting early accretion from narrow filaments, with accretion becoming more spherical as the filaments grow thicker than the halos).

It is frequently the case that we want to randomly draw radii (e.g. of satellite galaxies or dark matter particles) from the NFW distribution.  How would we go about doing this?

This is an example of sampling from a known probability distribution, a problem that comes up frequently.

NumPy already has a lot of distributions built in:
https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.random.html
and SciPy takes this to the next level
https://docs.scipy.org/doc/scipy/reference/stats.html

But what if your distribution is not in the list (e.g. our case)?  You have three basic options
1. [Rejection sampling](https://en.wikipedia.org/wiki/Rejection_sampling).
2. Do CDF inverse analytically
3. Do CDF inverse numerically/approximately

The first is often the quickest to get going, though not always the most numerically efficient.  

The other two are built upon the realization that for any $p(x)$ the cumulative distribution has a uniform distribution.  That means if I define $dy=p(x)\,dx$ then draw $y$ from a uniform distribution the implicitly defined $x(y)$ will have distribution $p(x)$.  If you can explicitly do the inverse function $x(y)$ then you should, and this method will be the best you can do.  If the NFW profile were a single power-law, then we would be in this situation.  But it's not.

Let's start with rejection sampling.

In [ ]:
def nfw(c):
    """Returns a single draw from an NFW of concentration 'c' using rejection sampling."""
    while True:
        x = np.random.uniform(size=1) * c
        r = np.random.uniform(size=1)
        # Note 4x/(1+x)^2 is x^2\rho normalized to unity at peak.
        if r< 4*x/(1+x)**2:
            return(x[0]/c)
    #
# Now time how long this takes.  Rejection sampling is
# fast in languages such as C/C++ or Fortran but tends
# to be slow in Python.  If this is not an issue, this
# is often the quickest-to-solution method.
%time nfw(7.0)

This appears to be the algorithm under the hood of the SciPy [rvs_ratio_uniforms](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rvs_ratio_uniforms.html#scipy.stats.rvs_ratio_uniforms) method, but it's not stated explicitly.

In [ ]:
# Let's look at the distribution -- obviously not how we'd do many
# draws in real life, but good enough for now ...
cc  = 7.0
%time rad = np.array([nfw(cc) for i in range(5000)])
# Compare the histogram to r^2\rho(r).
fig,ax = plt.subplots(1,1)
ax.hist(rad,bins=20)
r   = np.linspace(1e-2,1,100)
x   = r*cc
rho = 1400 * x**2 / x / (1+x)**2
ax.plot(r,rho)
ax.set_xlabel(r'$r/r_{\rm vir}$',fontsize=18)
ax.set_ylabel(r'$r^2\ \rho(r)$',fontsize=18)

So that worked and was pretty quick to code!  For this level of use waiting <1 sec is clearly perfectly okay, but it might not scale to millions of draws.

What about the other two methods?  Method (2) won't work for us.  It's actually quite easy to look at the mass enclosed within radius $x$ for an NFW profile (i.e. the CDF) since the integrals are elementary:
$$
  M(<x) \propto \ln(1+x)-\frac{x}{1+x}
$$
the difficulty lies in inverting this to get $x(M)$.  Sometimes the [Lagrange inversion formula](https://en.wikipedia.org/wiki/Lagrange_inversion_theorem) is helpful in conjunction with other tricks for power series manipulation.  In our particular case this won't work due to the structure of the NFW profile.

The simplest way around this is to built a table of the CDF then just interpolate it.

In [ ]:
Nt = 100 # Number of table entries.
cc = 7
xx = np.linspace(0,cc,Nt)
rr = xx/cc  # r/r_vir.
Mx = np.log(1+xx)-xx/(1+xx)
Mx/= Mx[-1]
#
fig,ax = plt.subplots(1,2,figsize=(10,4))
ax[0].plot(rr,Mx)
ax[0].set_xlabel(r'$r/r_{\rm vir}$',fontsize=18)
ax[0].set_ylabel(r'$M/M_{\rm vir}$',fontsize=18)
ax[1].plot(Mx,rr)
ax[1].set_xlabel(r'$M/M_{\rm vir}$',fontsize=18)
ax[1].set_ylabel(r'$r/r_{\rm vir}$',fontsize=18)

In [ ]:
# Now we can sample from this -- note the inversion of Mx and rr order.
xx  = np.random.uniform(size=5000)
%time rad = np.interp(xx,Mx,rr)
# Compare the histogram to r^2\rho(r).
fig,ax = plt.subplots(1,1)
ax.hist(rad,bins=20)
r   = np.linspace(1e-2,1,100)
x   = r*cc
rho = 1400 * x**2 / x / (1+x)**2
ax.plot(r,rho)
ax.set_xlabel(r'$r/r_{\rm vir}$',fontsize=18)
ax.set_ylabel(r'$r^2\ \rho(r)$',fontsize=18)

So that also worked well, and was almost two orders of magnitude faster because we used the library function np.interp to do the hard work (this is coded in C).  To be fair we probably should have compared this to a C coded version of rejection sampling -- but then wrapping that probably defeats the main advantage of rejection sampling: quick time-to-code.

In [ ]:
## Code to find the coefficiencts of a rational approximation to a tabulated function.
def find_rational(xi,fxi,A,B):
    """Finds the coefficients of the rational approximation to f(x) [see above]."""
    # First set up phi_{j\alpha}.  Can do this with cleverness, but let's
    # try to make the code more readable -- runtime isn't an issue.
    Ndata = len(xi)
    Nbase = A+B+1
    phi   = np.zeros( (Ndata,Nbase) )
    for i in range(Ndata):
        for a in range(Nbase):
            phi[i,a] = xi[i]**a if a<=A else (-fxi[i]*xi[i]**(a-A))
    # Now set up the "source" and solve for c.
    d = np.dot( fxi,phi )
    M = np.dot( phi.T,phi )
    c = np.dot( np.linalg.inv(M),d )
    num=np.poly1d(c[:A+1][::-1])
    den=np.poly1d(np.append(c[A+1:][::-1],1.0))
    return( (num,den) )
    #
num,den = find_rational(Mx,rr,3,3)
print(num)
print(den)
# Find the maximum error.
mm = np.linspace(0.05,1,500,endpoint=True)
ex = np.interp(mm,Mx,rr)
pa = num(mm)/den(mm)
err= np.abs((pa-ex)/ex)
print("Maximum error {:f}% at M={:f}".format(100*np.max(err),mm[np.argmax(err)]))

In [ ]:
# Compare the two graphically, just because ...
fig,ax = plt.subplots(1,1,figsize=(6,4))
ax.plot(Mx,rr,label='Table')
mm = np.linspace(0,1,50,endpoint=True)
ax.plot(mm,num(mm)/den(mm),label='Rational')
ax.set_xlabel(r'$M/M_{\rm vir}$',fontsize=18)
ax.set_ylabel(r'$r/r_{\rm vir}$',fontsize=18)

In [ ]:
# And look at our usual distribution.
xx  = np.random.uniform(size=5000)
%time rad = num(xx)/den(xx)
# Compare the histogram to r^2\rho(r).
fig,ax = plt.subplots(1,1)
ax.hist(rad,bins=20)
r   = np.linspace(1e-2,1,100)
x   = r*cc
rho = 1400 * x**2 / x / (1+x)**2
ax.plot(r,rho)
ax.set_xlabel(r'$r/r_{\rm vir}$',fontsize=18)
ax.set_ylabel(r'$r^2\ \rho(r)$',fontsize=18)

So this was actually even faster, and not very much more work!  The result would be a very portable piece of code.